In [1]:
import random
import json
import random
import time

In [2]:
VOWEL_COST = 250
LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
VOWELS = 'AEIOU'

In [3]:
# Write the WOFPlayer class definition (part A) here
class WOFPlayer:
    def __init__(self, name):
        self.name = name
        self.prizeMoney = 0
        self.prizes = []
    
    def addMoney(self, amt):
        self.prizeMoney += amt
    
    def goBankrupt(self):
        self.prizeMoney = 0

    def addPrize(self, prize):
        self.prizes.append(prize)
    
    def __str__(self):
        return '{} (${})'.format(self.name, self.prizeMoney)

In [4]:
# Write the WOFHumanPlayer class definition (part B) here
class WOFHumanPlayer(WOFPlayer):
    def getMove(self, category, obscuredPhrase, guessed):
        print('{} has ${}'.format(self.name, self.prizeMoney))
        print('Category:', category)
        print('Phrase:', obscuredPhrase)
        print('Guessed:', ', '.join(guessed))
        return input("Guess a letter, phrase, or type 'exit' or 'pass': ")

In [5]:
# Write the WOFComputerPlayer class definition (part C) here
class WOFComputerPlayer(WOFPlayer):
    SORTED_FREQUENCIES = 'ZQXJKVBPYGFWMUCLDRHSNIOATE'

    def __init__(self, name, difficulty):
        super().__init__(name)
        self.difficulty = difficulty
    
    def smartCoinFlip(self):
        return random.randint(1, 10) <= self.difficulty
    
    def getPossibleLetters(self, guessed):
        if self.prizeMoney >= VOWEL_COST:
            return [letter for letter in LETTERS if letter not in guessed]
        else:
            return [letter for letter in LETTERS if letter not in guessed and letter not in VOWELS]
    
    def getMove(self, category, obscuredPhrase, guessed):
        print('{} has ${}'.format(self.name, self.prizeMoney))
        print('Category:', category)
        print('Phrase:', obscuredPhrase)
        print('Guessed:', ', '.join(guessed))
        
        possible_letters = self.getPossibleLetters(guessed)
        
        if not possible_letters:
            return 'pass'
        
        if self.smartCoinFlip():
            return max(possible_letters, key=lambda letter: self.SORTED_FREQUENCIES.index(letter))
        else:
            return random.choice(possible_letters)

In [6]:
# Repeatedly asks the user for a number between min & max (inclusive)
def getNumberBetween(prompt, min, max):
    userinp = input(prompt) # ask the first time

    while True:
        try:
            n = int(userinp) # try casting to an integer
            if n < min:
                errmessage = 'Must be at least {}'.format(min)
            elif n > max:
                errmessage = 'Must be at most {}'.format(max)
            else:
                return n
        except ValueError: # The user didn't enter a number
            errmessage = '{} is not a number.'.format(userinp)

        # If we haven't gotten a number yet, add the error message
        # and ask again
        userinp = input('{}\n{}'.format(errmessage, prompt))

# Spins the wheel of fortune wheel to give a random prize
# Examples:
#    { "type": "cash", "text": "$950", "value": 950, "prize": "A trip to Ann Arbor!" },
#    { "type": "bankrupt", "text": "Bankrupt", "prize": false },
#    { "type": "loseturn", "text": "Lose a turn", "prize": false }

In [7]:
def spinWheel():
    with open("wheel.json", 'r') as f:
        wheel = json.loads(f.read())
        return random.choice(wheel)

# Returns a category & phrase (as a tuple) to guess
# Example:
#     ("Artist & Song", "Whitney Houston's I Will Always Love You")

In [8]:
def getRandomCategoryAndPhrase():
    with open("phrases.json", 'r') as f:
        phrases = json.loads(f.read())

        category = random.choice(list(phrases.keys()))
        phrase   = random.choice(phrases[category])
        return (category, phrase.upper())

# Given a phrase and a list of guessed letters, returns an obscured version
# Example:
#     guessed: ['L', 'B', 'E', 'R', 'N', 'P', 'K', 'X', 'Z']
#     phrase:  "GLACIER NATIONAL PARK"
#     returns> "_L___ER N____N_L P_RK"

In [9]:
def obscurePhrase(phrase, guessed):
    rv = ''
    for s in phrase:
        if (s in LETTERS) and (s not in guessed):
            rv = rv+'_'
        else:
            rv = rv+s
    return rv

In [10]:
# Returns a string representing the current state of the game
def showBoard(category, obscuredPhrase, guessed):
    return """
Category: {}
Phrase:   {}
Guessed:  {}""".format(category, obscuredPhrase, ', '.join(sorted(guessed)))

In [11]:
def requestPlayerMove(player, category, guessed):
    while True: # we're going to keep asking the player for a move until they give a valid one
        time.sleep(0.1) # added so that any feedback is printed out before the next prompt

        move = player.getMove(category, obscurePhrase(phrase, guessed), guessed)
        move = move.upper() # convert whatever the player entered to UPPERCASE
        if move == 'EXIT' or move == 'PASS':
            return move
        elif len(move) == 1: # they guessed a character
            if move not in LETTERS: # the user entered an invalid letter (such as @, #, or $)
                print('Guesses should be letters. Try again.')
                continue
            elif move in guessed: # this letter has already been guessed
                print('{} has already been guessed. Try again.'.format(move))
                continue
            elif move in VOWELS and player.prizeMoney < VOWEL_COST: # if it's a vowel, we need to be sure the player has enough
                    print('Need ${} to guess a vowel. Try again.'.format(VOWEL_COST))
                    continue
            else:
                return move
        else: # they guessed the phrase
            return move


In [12]:
# GAME LOGIC CODE
print('='*15)
print('WHEEL OF PYTHON')
print('='*15)
print('')

num_human = getNumberBetween('How many human players?', 0, 10)

# Create the human player instances
human_players = [WOFHumanPlayer(input('Enter the name for human player #{}'.format(i+1))) for i in range(num_human)]

num_computer = getNumberBetween('How many computer players?', 0, 10)

# If there are computer players, ask how difficult they should be
if num_computer >= 1:
    difficulty = getNumberBetween('What difficulty for the computers? (1-10)', 1, 10)

# Create the computer player instances
computer_players = [WOFComputerPlayer('Computer {}'.format(i+1), difficulty) for i in range(num_computer)]

players = human_players + computer_players

# No players, no game :(
if len(players) == 0:
    print('We need players to play!')
    raise Exception('Not enough players')

# category and phrase are strings.
category, phrase = getRandomCategoryAndPhrase()
# guessed is a list of the letters that have been guessed
guessed = []

# playerIndex keeps track of the index (0 to len(players)-1) of the player whose turn it is
playerIndex = 0

# will be set to the player instance when/if someone wins
winner = False

WHEEL OF PYTHON



How many human players? 1
Enter the name for human player #1 Awaiz
How many computer players? 1
What difficulty for the computers? (1-10) 4


In [13]:
while True:
    player = players[playerIndex]
    wheelPrize = spinWheel()

    print('')
    print('-'*15)
    print(showBoard(category, obscurePhrase(phrase, guessed), guessed))
    print('')
    print('{} spins...'.format(player.name))
    time.sleep(2) # pause for dramatic effect!
    print('{}!'.format(wheelPrize['text']))
    time.sleep(1) # pause again for more dramatic effect!

    if wheelPrize['type'] == 'bankrupt':
        player.goBankrupt()
    elif wheelPrize['type'] == 'loseturn':
        pass # do nothing; just move on to the next player
    elif wheelPrize['type'] == 'cash':
        move = requestPlayerMove(player, category, guessed)
        if move == 'EXIT': # leave the game
            print('Until next time!')
            break
        elif move == 'PASS': # will just move on to next player
            print('{} passes'.format(player.name))
        elif len(move) == 1: # they guessed a letter
            guessed.append(move)

            print('{} guesses "{}"'.format(player.name, move))

            if move in VOWELS:
                player.prizeMoney -= VOWEL_COST

            count = phrase.count(move) # returns an integer with how many times this letter appears
            if count > 0:
                if count == 1:
                    print("There is one {}".format(move))
                else:
                    print("There are {} {}'s".format(count, move))

                # Give them the money and the prizes
                player.addMoney(count * wheelPrize['value'])
                if wheelPrize['prize']:
                    player.addPrize(wheelPrize['prize'])

                # all of the letters have been guessed
                if obscurePhrase(phrase, guessed) == phrase:
                    winner = player
                    break

                continue # this player gets to go again

            elif count == 0:
                print("There is no {}".format(move))
        else: # they guessed the whole phrase
            if move == phrase: # they guessed the full phrase correctly
                winner = player

                # Give them the money and the prizes
                player.addMoney(wheelPrize['value'])
                if wheelPrize['prize']:
                    player.addPrize(wheelPrize['prize'])

                break
            else:
                print('{} was not the phrase'.format(move))

    # Move on to the next player (or go back to player[0] if we reached the end)
    playerIndex = (playerIndex + 1) % len(players)

if winner:
    # In your head, you should hear this as being announced by a game show host
    print('{} wins! The phrase was {}'.format(winner.name, phrase))
    print('{} won ${}'.format(winner.name, winner.prizeMoney))
    if len(winner.prizes) > 0:
        print('{} also won:'.format(winner.name))
        for prize in winner.prizes:
            print('    - {}'.format(prize))
else:
    print('Nobody won. The phrase was {}'.format(phrase))


---------------

Category: Fictional Character
Phrase:   _______ _____
Guessed:  

Awaiz spins...
$2500!
Awaiz has $0
Category: Fictional Character
Phrase: _______ _____
Guessed: 


Guess a letter, phrase, or type 'exit' or 'pass':  pass


Awaiz passes

---------------

Category: Fictional Character
Phrase:   _______ _____
Guessed:  

Computer 1 spins...
$800!
Computer 1 has $0
Category: Fictional Character
Phrase: _______ _____
Guessed: 
Computer 1 guesses "T"
There is no T

---------------

Category: Fictional Character
Phrase:   _______ _____
Guessed:  T

Awaiz spins...
$650!
Awaiz has $0
Category: Fictional Character
Phrase: _______ _____
Guessed: T


Guess a letter, phrase, or type 'exit' or 'pass':  pass


Awaiz passes

---------------

Category: Fictional Character
Phrase:   _______ _____
Guessed:  T

Computer 1 spins...
$600!
Computer 1 has $0
Category: Fictional Character
Phrase: _______ _____
Guessed: T
Computer 1 guesses "Y"
There is no Y

---------------

Category: Fictional Character
Phrase:   _______ _____
Guessed:  T, Y

Awaiz spins...
$900!
Awaiz has $0
Category: Fictional Character
Phrase: _______ _____
Guessed: T, Y


...

Awaiz passes

---------------

Category: Fictional Character
Phrase:   IN_IANA _ONES
Guessed:  A, E, F, G, H, I, K, L, M, N, O, R, S, T, U, Y, Z

Computer 1 spins...
$950!
Computer 1 has $5950
Category: Fictional Character
Phrase: IN_IANA _ONES
Guessed: T, Y, G, F, N, E, I, U, A, M, O, Z, S, H, K, L, R
Computer 1 guesses "P"
There is no P

---------------

Category: Fictional Character
Phrase:   IN_IANA _ONES
Guessed:  A, E, F, G, H, I, K, L, M, N, O, P, R, S, T, U, Y, Z

Awaiz spins...
$950!
Awaiz has $0
Category: Fictional Character
Phrase: IN_IANA _ONES
Guessed: T, Y, G, F, N, E, I, U, A, M, O, Z, S, H, K, L, R, P


Guess a letter, phrase, or type 'exit' or 'pass':  pass


Awaiz passes

---------------

Category: Fictional Character
Phrase:   IN_IANA _ONES
Guessed:  A, E, F, G, H, I, K, L, M, N, O, P, R, S, T, U, Y, Z

Computer 1 spins...
$600!
Computer 1 has $5950
Category: Fictional Character
Phrase: IN_IANA _ONES
Guessed: T, Y, G, F, N, E, I, U, A, M, O, Z, S, H, K, L, R, P
Computer 1 guesses "J"
There is one J

---------------

Category: Fictional Character
Phrase:   IN_IANA JONES
Guessed:  A, E, F, G, H, I, J, K, L, M, N, O, P, R, S, T, U, Y, Z

Computer 1 spins...
Lose a turn!

---------------

Category: Fictional Character
Phrase:   IN_IANA JONES
Guessed:  A, E, F, G, H, I, J, K, L, M, N, O, P, R, S, T, U, Y, Z

Awaiz spins...
One Million!
Awaiz has $0
Category: Fictional Character
Phrase: IN_IANA JONES
Guessed: T, Y, G, F, N, E, I, U, A, M, O, Z, S, H, K, L, R, P, J


Guess a letter, phrase, or type 'exit' or 'pass':  pass


Awaiz passes

---------------

Category: Fictional Character
Phrase:   IN_IANA JONES
Guessed:  A, E, F, G, H, I, J, K, L, M, N, O, P, R, S, T, U, Y, Z

Computer 1 spins...
One Million!
Computer 1 has $6550
Category: Fictional Character
Phrase: IN_IANA JONES
Guessed: T, Y, G, F, N, E, I, U, A, M, O, Z, S, H, K, L, R, P, J
Computer 1 guesses "X"
There is no X

---------------

Category: Fictional Character
Phrase:   IN_IANA JONES
Guessed:  A, E, F, G, H, I, J, K, L, M, N, O, P, R, S, T, U, X, Y, Z

Awaiz spins...
Lose a turn!

---------------

Category: Fictional Character
Phrase:   IN_IANA JONES
Guessed:  A, E, F, G, H, I, J, K, L, M, N, O, P, R, S, T, U, X, Y, Z

Computer 1 spins...
900!
Computer 1 has $6550
Category: Fictional Character
Phrase: IN_IANA JONES
Guessed: T, Y, G, F, N, E, I, U, A, M, O, Z, S, H, K, L, R, P, J, X
Computer 1 guesses "D"
There is one D
Computer 1 wins! The phrase was INDIANA JONES
Computer 1 won $7450
